# Part II - KES

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
import math

In [3]:
ALPHABET = string.ascii_uppercase
ALPHABET_OFFSETED = {c: ord(c) - ord('A') for c in ALPHABET}

In [4]:
enc_msg = "HSQQD XHDRP YFKWV HNHDL OULLQ DDWVW BDWWA RJULS".upper().replace(" ", "")
enc_msg

'HSQQDXHDRPYFKWVHNHDLOULLQDDWVWBDWWARJULS'

In [5]:
# phone book keys of the day
name, streetname, housenumber, phonenr = "John Smith", "Lagardsveien", 3, "51634782"

### Task 1. Present your cryptanalysis on this KES.

#### TODO

### Task 2. What is the clear-text of the message above?

In [6]:
test_message = "testing the ciphers"
test_key = "key"

#### Vigenere Cipher

In [7]:
def vigener_cipher(text, key, mode = 'encrypt'):
    message = ""
    text = text.upper().replace(" ", "").replace("\n", "")
    key = key.upper()

    for i in range(len(text)):
        if mode == "encrypt":
            message += ALPHABET[(ord(text[i]) + ord(key[i % len(key)])) % 26]
        else:
            message += ALPHABET[(ord(text[i]) - ord(key[i % len(key)])) % 26]
    return message


In [8]:
test_enc_vigener = vigener_cipher(test_message, test_key)
test_dec_vigener = vigener_cipher(test_enc_vigener, test_key, mode = 'decrypt')
print(test_enc_vigener, test_dec_vigener)

DIQDMLQXFOGGZLCBW TESTINGTHECIPHERS


#### Caesar Cipher

In [9]:
def ceasar_cipher(text, key, mode="encrypt"):
    message = ""
    text = text.upper().replace(" ", "").replace("\n", "")
    cipher_alpha = ALPHABET[key:] + ALPHABET[:key] # shift the alphabet by key positions

    # mapping between the two alphabets
    alpha_dict = dict(zip(cipher_alpha, ALPHABET)) if mode == "decrypt" else dict(zip(ALPHABET, cipher_alpha))
    for c in text:
        message += alpha_dict[c]
    return message

In [10]:
test_shift = 3
test_enc_ceasar = ceasar_cipher(test_message, test_shift)
test_dec_ceasar = ceasar_cipher(test_enc_ceasar, test_shift, mode="decrypt")
print(test_enc_ceasar, test_dec_ceasar)

WHVWLQJWKHFLSKHUV TESTINGTHECIPHERS


#### Transposition Cipher

In [11]:
def transposition_cipher(text, key, mode="encrypt"):
    message = ""
    text = text.upper().replace(" ", "").replace("\n", "")

    cols = len(key) # number of columns
    rows = math.ceil(len(text) / cols) # number of rows
    empty_spaces = (cols * rows) - len(text) # number of empty spaces

    # create transposition matrix
    matrix = np.array([[""] * cols] * rows)
    count = 0
    if mode == "encrypt":
        for i in range(len(text)):
            for j in range(cols):
                if count < len(text):
                    matrix[i, j]= text[count]
                    count += 1
        for i in key:
            for j in range(rows):
                col = int(i) - 1
                if matrix[j, col] != "":
                    message += matrix[j, col]
    else:
        for i in key:
            col = int(i) - 1
            for j in range(rows):
                if j < rows - 1:
                    matrix[j, col] = text[count]
                    count += 1
                    continue
                if col < (cols - empty_spaces):
                    matrix[j, col] = text[count]
                    count += 1

        for i in range(rows):
            for j in range(cols):
                if matrix[i, j] != "":
                    message += matrix[i, j]
    return message

In [12]:
test_enc_transposition = transposition_cipher(test_message, phonenr)
test_dec_transposition = transposition_cipher(test_enc_transposition, phonenr, mode="decrypt")
print(test_enc_transposition, test_dec_transposition)

IPTHSNHSCTIGETREE TESTINGTHECIPHERS


### Task 3. How would you improve this KES?

...

### Task 4. Implement your improved KES and encrypt the same message

In [17]:
def KES_cipher(text, street_name, house_number, phone_nr, mode="encrypt", withimprovments=False):
    message = ""
    text = text.upper().replace(" ", "").replace("\n", "")

    if mode == "encrypt":
        message = ceasar_cipher(text, house_number)
        message = transposition_cipher(message, phone_nr)
        if withimprovments:
            message = vigener_cipher(message, street_name)
    else:
        if withimprovments:
            message = vigener_cipher(text, street_name, mode="decrypt")
        if withimprovments:
            message = transposition_cipher(message, phone_nr, mode="decrypt")
        else:
            message = transposition_cipher(text, phone_nr, mode="decrypt")
        message = ceasar_cipher(message, house_number, mode="decrypt")
    return message


In [14]:
test_enc_KES = KES_cipher(test_message, streetname, housenumber, phonenr)
test_dec_KES = KES_cipher(test_enc_KES, streetname, housenumber, phonenr, mode="decrypt")
print(test_enc_KES, test_dec_KES)

WSCKMTCQJEPWSWAHY TESTINGTHECIPHERS


In [21]:
dec_KES = KES_cipher(enc_msg, streetname, housenumber, phonenr, mode="decrypt")
hmm = ceasar_cipher(transposition_cipher(enc_msg, phonenr, mode="decrypt"), housenumber, mode="decrypt")
print(dec_KES, hmm, dec_KES == hmm)

UOELEVAYEGKRPCAAAREINHTTOIAINTSTMPINASTX UOELEVAYEGKRPCAAAREINHTTOIAINTSTMPINASTX True


WTF

### Task 5. Webserver using improved KES

In [16]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def index():
    return "To encrypt a message: /encrypt?text=mymessage. <br> To decrypt a cipher: /decrypt?cipher=mycipher'"

@app.route('/encrypt')
def encrypt():
    message = request.args.get('text')
    return "Cipher" + KES_cipher(message, streetname, housenumber, phonenr)

@app.route('/decrypt')
def decrypt():
    message = request.args.get('cipher')
    return "Plaintext" + KES_cipher(message, streetname, housenumber, phonenr, mode="decrypt")

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [22]:
ord('A')

65